# Setup code:

In [ ]:
import urllib.parse as urlparse
from aiidalab_widgets_base import AiiDACodeSetup

In [ ]:
parsed_url = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)
args = {}
for key, value in parsed_url.items():
    args[key] = '\n'.join(value) if isinstance(value,(tuple,list)) else value

In [ ]:
code = AiiDACodeSetup(**args)
display(code)